In [1]:
import sys
sys.path.insert(1, 
       '/Users/jeanettemumford/Dropbox/Research/Projects/RT_sims/Code')
import scipy.stats
import matplotlib.pyplot as plt
import numpy as np
from functions import *
from scipy.stats import gamma, exponnorm
import time 
from nilearn.glm.first_level.design_matrix import _cosine_drift
import pingouin as pg


/usr/local/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
/usr/local/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


In [2]:
rt_grinband_shift = gamma.rvs(a=1.7, loc=.5, scale=.49, size=10000)
shape_grinband_shift, mu_grinband_shift, sigma_grinband_shift =\
     exponnorm.fit(rt_grinband_shift*1000)
inv_lambda_grinband_shift = np.multiply(sigma_grinband_shift,
                                        shape_grinband_shift)
n_trials = 30
scan_length = 225
repetition_time = 1

mu_expnorm = mu_grinband_shift
lam_expnorm = 1 / inv_lambda_grinband_shift
sigma_expnorm = sigma_grinband_shift

max_rt = 8000
min_rt = 50
event_duration = .5
beta_scales_yes = 5
beta_scales_no = 30
nsim = 100
center_rt = True

win_sub_noise_sd_range_scales_yes = [1.2, 1.75,  3, 4, 10]
win_sub_noise_sd_range_scales_no = [.25,  .35, .5,  .75, 3]

nsim_pow = 5000
ISI_min_max_vec = [(1, 3), (2, 5), (3, 6), (4, 7)]

mu_expnorm = mu_grinband_shift
lam_expnorm = 1 / inv_lambda_grinband_shift
sigma_expnorm = sigma_grinband_shift

hp_filter = True



In [3]:
# Does sim_fit_sub die if I run it by itself?
# Trying to determine if it is this function or what I do with the 
# output of this function in calc_win_sub_pow_range

nsim_check = 200000
ISI_min = 3
ISI_max = 6
sd_ind = 1
start_time = time.time()
for i in range(nsim_check):
    if i%1000 == 0:
        print(i)
        print(start_time - time.time())
        start_time = time.time()
    regressors, _ = make_regressors_one_trial_type(n_trials, scan_length, 
                                   repetition_time, mu_expnorm, 
                                   lam_expnorm, sigma_expnorm,
                                   max_rt, min_rt, event_duration, ISI_min, 
                                   ISI_max, center_rt)
    #models = make_design_matrices(regressors)

0
-0.0005319118499755859
1000
-6.124389886856079
2000
-6.221172094345093
3000
-6.051740884780884
4000
-5.9263739585876465
5000
-5.938540935516357
6000
-5.955796003341675
7000
-6.043513298034668
8000
-7.092565059661865
9000
-6.350291967391968
10000
-5.628414869308472
11000
-5.603831052780151
12000
-5.964833974838257
13000
-5.6046929359436035
14000
-5.6889519691467285
15000
-5.895969867706299
16000
-5.883195161819458
17000
-6.251052141189575
18000
-5.944021940231323
19000
-5.973551988601685
20000
-7.790924072265625
21000
-5.929134130477905
22000
-5.651681184768677
23000
-5.697509288787842
24000
-5.980211973190308
25000
-5.729849100112915
26000
-5.602617025375366
27000
-5.7787766456604
28000
-6.096846103668213
29000
-5.925615072250366
30000
-5.968402147293091
31000
-6.024040937423706
32000
-5.983017206192017
33000
-5.966516017913818
34000
-6.5857930183410645
35000
-7.066650152206421
36000
-7.132980108261108
37000
-7.432942867279053
38000
-7.433694124221802
39000
-6.06093692779541
40000
-5

KeyboardInterrupt: 

In [ ]:

output_unmod_beta_power_grin_hp_yes, output_rtmod_beta_power_grin_hp_yes = \
       calc_win_sub_pow_range(n_trials, scan_length, repetition_time,
                              mu_expnorm, lam_expnorm, sigma_expnorm,
                              max_rt, min_rt, event_duration, ISI_min_max_vec,
                              win_sub_noise_sd_range_scales_yes,
                              win_sub_noise_sd_range_scales_no, center_rt,
                              beta_scales_yes, beta_scales_no, hp_filter,
                              nsim_pow)

In [ ]:
output_unmod_beta_power_grin_hp_yes, output_rtmod_beta_power_grin_hp_yes = \
       calc_win_sub_pow_range2(n_trials, scan_length, repetition_time,
                              mu_expnorm, lam_expnorm, sigma_expnorm,
                              max_rt, min_rt, event_duration, ISI_min_max_vec,
                              win_sub_noise_sd_range_scales_yes,
                              win_sub_noise_sd_range_scales_no, center_rt,
                              beta_scales_yes, beta_scales_no, hp_filter,
                              nsim_pow)